### Libraries

In [145]:
# import required libraries
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer import Pitch, VerticalPitch
import matplotlib_inline
from matplotlib import cm
import matplotlib_inline
import seaborn as sns
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

### Convert event data from .json match file into .csv

In [132]:
# extract all match names in directory as a list 
files = os.listdir('matches_2021_json')

# avoid last file 'players.json' indexed as '-1'
for i in files[:-1]:
    with open('matches_2021_json\\' + i, 'r') as f:
        data = json.load(f)
   
    # convert list of dictionaries into data frame 
    events = pd.DataFrame(data['events'])
    
    # reset indices
    events = events.reset_index(drop = True)
    
    # rearrange columns
    # since only single match has less than 23 columns which is '397511822166547-Halmstad-DegerforsIF'
    if len(events.columns) < 23:
        col_rearr = ['external_id', 'team', 'player', 'next_player', 'xpos', 'ypos', 'xdest', 'ydest', 'start_time', 'end_time',
                    'action', 'xg', 'xt', 'xp','game_time', 'header', 'penalty', 'throw_in']
    else:
        col_rearr = ['external_id','team', 'player', 'next_player', 'xpos', 'ypos', 'xdest', 'ydest', 'start_time', 
                     'end_time','action', 'xg', 'xt', 'xp','game_time', 'header', 'penalty', 'throw_in', 'attack_type', 
                    'corner_outcome', 'foot_used', 'one_touch', 'goal_mouth']
    events = events[col_rearr]
    
    # replace all None values with NaN (missing value)
    events.fillna(value = np.nan, inplace = True)
    
    # save dataframe 
    file_name = i.split('.')[0]
    events.to_csv('matches_2021_csv\\' + file_name + '.csv', index = False)

### Minutes Played by Players data

In [13]:
# load info 'how much players played in each game'
with open('matches_2021_json\\allsvenskan, 2021-players.json', 'r') as f:
    data2 = json.load(f)

# aggregate results into data frame
playing_time = pd.DataFrame()
for i in data2:
    single_col = {'player_name':i['name'], 'team_name':i['team'],
                  'minutes_played':sum(i['minutes']['62163887d0736a3c7964ff4b'].values())}
    playing_time = pd.concat([playing_time, pd.DataFrame([single_col])])

playing_time = playing_time.reset_index(drop = True)

In [385]:
playing_time.head()

NameError: name 'playing_time' is not defined

In [ ]:
# Incorrect minutes data for players. Examples are below:
# Justin Salmon, Degerfors
# Ali Youssef, Hacken

In [ ]:
# quick links
# https://fbref.com/en/comps/29/11002/2021-Allsvenskan-Stats
# https://fbref.com/en/comps/29/11002/stats/2021-Allsvenskan-Stats#all_stats_standard

### Refining FBref data
`Match results` with all match scores and `player_stats` with minutes played by players

In [67]:
# load fbfref data
match_results = pd.read_csv("match_results.csv")
# create column indices to be removed
rm_col_ind = np.r_[0:5, 6, -5:0]
# remove columns
match_results = match_results.drop(columns = match_results.columns[rm_col_ind], axis = 1)
# make all column names lowercase
match_results = match_results.rename(columns = str.lower)
# save refined .csv file
match_results.to_csv('match_results.csv', index = False)

In [49]:
# load fbref data 
player_stats = pd.read_csv('player_stats.csv')
# create column indices to be removed
rm_col_ind = np.r_[0, 2, -1]
# remove columns
player_stats = player_stats.drop(columns = player_stats.columns[rm_col_ind], axis = 1)
# make all column names lowercase
player_stats = player_stats.rename(columns = str.lower)
player_stats.to_csv('player_stats.csv', index = False)

### Save all match events into a single dataframe

In [116]:
csv_files = os.listdir('matches_2021_csv/')

all_match_events = pd.DataFrame()
for i in csv_files:
    df = pd.read_csv('matches_2021_csv/'+ i)
    all_match_events = pd.concat([all_match_events, df])
all_match_events.to_csv('all_match_events.csv', index = False)    

In [117]:
all_match_events.shape

(409713, 23)

In [118]:
all_match_events

,external_id,team,player,next_player,xpos,ypos,xdest,ydest,start_time,end_time,...,xp,game_time,header,penalty,throw_in,attack_type,corner_outcome,foot_used,one_touch,goal_mouth
0,0,BK Häcken,Tobias Heintz,Alexander Faltsetas,51,50,37.0,56.0,0,3,...,0.882111,1,False,False,False,NaN,NaN,NaN,NaN,NaN
1,1,BK Häcken,Alexander Faltsetas,Godswill Ekpolo,37,52,40.0,9.0,1,4,...,0.772046,1,False,False,False,NaN,NaN,NaN,NaN,NaN
2,2,BK Häcken,Godswill Ekpolo,Joona Toivio,41,7,27.0,25.0,7,10,...,0.906022,1,False,False,False,NaN,NaN,NaN,NaN,NaN
3,3,BK Häcken,Joona Toivio,Johan Hammar,25,29,25.0,68.0,10,13,...,0.895334,1,False,False,False,NaN,NaN,NaN,NaN,NaN
4,4,BK Häcken,Johan Hammar,Joona Toivio,31,63,31.0,33.0,15,18,...,0.867001,1,False,False,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1799,1799,AIK,Zachary Elbouzedi,NaN,90,14,NaN,NaN,5597,5607,...,NaN,94,False,False,False,NaN,NaN,NaN,NaN,NaN
1800,1800,AIK,Zachary Elbouzedi,NaN,92,13,NaN,NaN,5598,5603,...,NaN,94,False,False,False,NaN,NaN,NaN,NaN,NaN
1801,1801,AIK,Zachary Elbouzedi,NaN,66,16,92.0,13.0,5592,5603,...,NaN,94,False,False,False,NaN,NaN,NaN,NaN,NaN
1802,1802,Varbergs BoIS FC,Philip Mårtensson,Joakim Lindner,6,50,24.0,22.0,5600,5610,...,NaN,94,False,False,False,NaN,NaN,NaN,NaN,NaN


In [ ]:
# https://stackoverflow.com/questions/26139423/plot-different-color-for-different-categorical-levels-using-matplotlib

### Derive xG results per match

In [237]:
json_files = os.listdir('matches_2021_json/')

In [393]:
# Create data frame with results of all matches
xg_results = pd.DataFrame()

# avoid last file 'players.json'
for i in json_files[:-1]:
    with open('matches_2021_json/' + i, 'r') as f:
        data = json.load(f)

    single_col = {'home':data['homeTeam'], 'away':data['visitingTeam'], 
              'home_xg':json.loads(data['xG'])[0], 'away_xg':json.loads(data['xG'])[1]}

    xg_results = pd.concat([xg_results, pd.DataFrame([single_col])])
xg_results = xg_results.reset_index(drop = True)

In [394]:
xg_results.head()

,home,away,home_xg,away_xg
0,BK Häcken,Örebro,3.136022,0.946330
1,Kalmar FF,Östersund,0.811020,0.156506
2,Degerfors IF,IFK Göteborg,0.652445,0.908759
3,IF Elfsborg,Varbergs BoIS FC,0.381496,1.120826
4,Örebro,AIK,1.274922,3.121992


In [272]:
# sanity check of xG results from .csv file
csv_files = os.listdir('matches_2021_csv/')
df = pd.read_csv('matches_2021_csv/' + csv_files[0])
df[(df['action'] == 'Shot') | (df['action'] == 'Goal')][['action', 'team', 'xg']].groupby(by = 'team').sum()

,xg
team,
BK Häcken,3.136022
Örebro,0.946330


### Replace team names in fbref data with team names in playmakerAI data

In [289]:
xg_names = xg_results['home_team'].unique()
xg_results['home_team'].unique()

array(['BK Häcken', 'Kalmar FF', 'Degerfors IF', 'IF Elfsborg', 'Örebro',
       'Malmö FF', 'Östersund', 'IK Sirius FK', 'Hammarby',
       'IFK Norrköping FK', 'Halmstad', 'IFK Göteborg', 'AIK',
       'Mjällby AIF', 'Djurgården', 'Varbergs BoIS FC'], dtype=object)

In [290]:
match_names = match_results['home'].unique()
match_results['home'].unique()

array(['Malmö', 'Örebro', 'Halmstad', 'Mjällby', 'Elfsborg', 'Norrköping',
       'AIK Stockholm', 'Kalmar', 'Hammarby', 'Degerfors', 'Östersund',
       'Häcken', 'Djurgården', 'Sirius', 'Göteborg', 'Varberg'],
      dtype=object)

In [344]:
xg_ls = []
for i in xg_names:
    xg_ls.append(i.split(' '))

xg_ls

[['BK', 'Häcken'],
 ['Kalmar', 'FF'],
 ['Degerfors', 'IF'],
 ['IF', 'Elfsborg'],
 ['Örebro'],
 ['Malmö', 'FF'],
 ['Östersund'],
 ['IK', 'Sirius', 'FK'],
 ['Hammarby'],
 ['IFK', 'Norrköping', 'FK'],
 ['Halmstad'],
 ['IFK', 'Göteborg'],
 ['AIK'],
 ['Mjällby', 'AIF'],
 ['Djurgården'],
 ['Varbergs', 'BoIS', 'FC']]

In [359]:
'AIK'.join(" ")

' '

In [381]:
names_dict = {}
for i in xg_ls:
    for j in i:
        if j in match_names:
            full_name = ' '.join(i)
            short_name = list(match_names[match_names == j])[0]
            names_dict[full_name] = short_name

# special cases 'AIK Stockholm' and 'Varbergs'        
names_dict['AIK'] = 'AIK Stockholm'
names_dict['Varbergs BoIS FC'] = 'Varberg' 

# reverse keys and values in
names_dict = dict((value, key) for key, value in names_dict.items())
names_dict

{'Häcken': 'BK Häcken',
 'Kalmar': 'Kalmar FF',
 'Degerfors': 'Degerfors IF',
 'Elfsborg': 'IF Elfsborg',
 'Örebro': 'Örebro',
 'Malmö': 'Malmö FF',
 'Östersund': 'Östersund',
 'Sirius': 'IK Sirius FK',
 'Hammarby': 'Hammarby',
 'Norrköping': 'IFK Norrköping FK',
 'Halmstad': 'Halmstad',
 'Göteborg': 'IFK Göteborg',
 'Mjällby': 'Mjällby AIF',
 'Djurgården': 'Djurgården',
 'AIK Stockholm': 'AIK',
 'Varberg': 'Varbergs BoIS FC'}

In [384]:
# replace team names in fbref data with original names from playmaker AI
match_results['home'] = match_results['home'].apply(lambda x: names_dict[x])
match_results['away'] = match_results['away'].apply(lambda x: names_dict[x])
match_results.head()

,wk,date,time,home,homegoals,away,awaygoals
0,1,2021-04-10,15:00,Malmö FF,3,Hammarby,2
1,1,2021-04-10,17:30,Örebro,0,IFK Göteborg,0
2,1,2021-04-11,15:00,Halmstad,1,BK Häcken,0
3,1,2021-04-11,15:00,Mjällby AIF,0,Varbergs BoIS FC,0
4,1,2021-04-11,17:30,IF Elfsborg,0,Djurgården,2


### Merge FBref and PlaymakerAI Data
`Match scores` from FBref and `match xg scores` from PlaymakerAI

In [410]:
# PlaymakerAI data with xg scores per match
xg_results.head()

,home,away,home_xg,away_xg
0,BK Häcken,Örebro,3.136022,0.946330
1,Kalmar FF,Östersund,0.811020,0.156506
2,Degerfors IF,IFK Göteborg,0.652445,0.908759
3,IF Elfsborg,Varbergs BoIS FC,0.381496,1.120826
4,Örebro,AIK,1.274922,3.121992


In [388]:
# FBref data with match scores
match_results.head()

,wk,date,time,home,homegoals,away,awaygoals
0,1,2021-04-10,15:00,Malmö FF,3,Hammarby,2
1,1,2021-04-10,17:30,Örebro,0,IFK Göteborg,0
2,1,2021-04-11,15:00,Halmstad,1,BK Häcken,0
3,1,2021-04-11,15:00,Mjällby AIF,0,Varbergs BoIS FC,0
4,1,2021-04-11,17:30,IF Elfsborg,0,Djurgården,2


In [427]:
# merge two dataframes on column names 'home' and 'away'
merged_results = match_results.merge(right = xg_results, on = ['home', 'away'])
merged_results.head()

,wk,date,time,home,homegoals,away,awaygoals,home_xg,away_xg
0,1,2021-04-10,15:00,Malmö FF,3,Hammarby,2,3.081973,0.284890
1,1,2021-04-10,17:30,Örebro,0,IFK Göteborg,0,0.630045,0.747871
2,1,2021-04-11,15:00,Halmstad,1,BK Häcken,0,1.014930,0.405163
3,1,2021-04-11,15:00,Mjällby AIF,0,Varbergs BoIS FC,0,1.517195,0.707990
4,1,2021-04-11,17:30,IF Elfsborg,0,Djurgården,2,0.841527,0.221132


In [424]:
# save .csv file
merged_results.to_csv('merged_match_results.csv', index = False)